FACIAL EMOTION RECOGNITION (DEEP LEARNING CNN)


IMPLEMENTATION

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import kagglehub
import shutil
import pathlib
import tarfile


# Download LFW People dataset
dataset_path = kagglehub.dataset_download("atulanandjha/lfwpeople")

print(f"Dataset path: {dataset_path}")

Using Colab cache for faster access to the 'lfwpeople' dataset.
Dataset path: /kaggle/input/lfwpeople


The images are usually inside a subfolder named 'lfw_funneled'

The dataset contains a 'lfw-funneled.tgz' file that needs to be extracted manually

so i did manually extracted that the dataset_path was likely reassigned to a list. Revert to the expected string path.

In [ ]:
if isinstance(dataset_path, list):
    print(f"Warning: dataset_path was a list {dataset_path}. Reverting to the expected string path: /kaggle/input/lfwpeople")
    dataset_path = "/kaggle/input/lfwpeople"

tgz_path = os.path.join(dataset_path, "lfw-funneled.tgz")
extract_path = os.path.join("/content/", "lfw_funneled_extracted")

if os.path.exists(tgz_path):
    print(f"Found compressed file: {tgz_path}")
    if not os.path.exists(extract_path):
        print("Extracting inner .tgz file... (This may take a moment)")
        with tarfile.open(tgz_path, "r:gz") as tar:
            tar.extractall(path=extract_path)
        print("Extraction complete.")
    else:
        print("Files already extracted.")

    # The images are inside the extracted folder
    # usually structure is: extracted/lfw_funneled/{person_name}/{image.jpg}
    data_dir = pathlib.Path(extract_path) / "lfw_funneled"
else:
    print("No .tgz file found, checking for folders directly...")
    data_dir = pathlib.Path(dataset_path) / "lfw_funneled"

if not data_dir.exists():
    data_dir = pathlib.Path(dataset_path)

print(f"Looking for images in: {data_dir}")

Found compressed file: /kaggle/input/lfwpeople/lfw-funneled.tgz
Files already extracted.
Looking for images in: /content/lfw_funneled_extracted/lfw_funneled


PREPROCESSING (CRITICAL STEP)

LFW has 5,700 people, but most have only 1 image.
A CNN cannot learn from 1 image.
We must FILTER the dataset to keep only people with many images.

this code is going to be only keep people with 70+ photos

In [ ]:
MIN_IMAGES_REQUIRED = 50
FILTERED_DIR = pathlib.Path("./filtered_dataset")

if os.path.exists(FILTERED_DIR):
    shutil.rmtree(FILTERED_DIR)
os.makedirs(FILTERED_DIR)

Loop through every person folder and check the count

also data_dir should be make sure it exists before iterating

In [ ]:
selected_classes = []

if not data_dir.exists():
    print(f"CRITICAL ERROR: Could not find image directory at {data_dir}")
    exit()

count_found = 0
for person_dir in data_dir.iterdir():
    if person_dir.is_dir():
        count_found += 1
        # lfw usually has jpg images
        images = list(person_dir.glob('*.jpg'))
        num_images = len(images)

        if num_images >= MIN_IMAGES_REQUIRED:
            print(f"Keeping {person_dir.name}: {num_images} images")
            selected_classes.append(person_dir.name)
            # Copy folder to our new filtered directory
            shutil.copytree(person_dir, FILTERED_DIR / person_dir.name)

Keeping Tony_Blair: 144 images
Keeping Ariel_Sharon: 77 images
Keeping John_Ashcroft: 53 images
Keeping Hugo_Chavez: 71 images
Keeping Jacques_Chirac: 52 images
Keeping Colin_Powell: 236 images
Keeping Junichiro_Koizumi: 60 images
Keeping Gerhard_Schroeder: 109 images
Keeping Donald_Rumsfeld: 121 images
Keeping Jean_Chretien: 55 images
Keeping George_W_Bush: 530 images
Keeping Serena_Williams: 52 images


In [ ]:
print(f"Scanned {count_found} folders.")
print(f"--- Filtering Complete. Selected {len(selected_classes)} people. ---")

if len(selected_classes) == 0:
    print("ERROR: No classes selected. Please check the path or lower MIN_IMAGES_REQUIRED.")
    exit()

Scanned 5749 folders.
--- Filtering Complete. Selected 12 people. ---


Parameters

LFW images are larger, so we use 100x100

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 100
IMG_WIDTH = 100

DATA SPLITTING

 Since LFW doesn't have train/test folders, we split them automatically (80% train, 20% val)

 Faces need color often but i can do grayscale too. Let's use RGB

Split Training Data

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    FILTERED_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='rgb'
)

Found 1560 files belonging to 12 classes.
Using 1248 files for training.


Split Testing Data

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    FILTERED_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='rgb'
)

Found 1560 files belonging to 12 classes.
Using 312 files for validation.


ii did part of peformance boosting and data augmentation.

so the code for performance boosting to make the cache data in memory to make training faster

Also the augmentation, I create new variations of images (flips, rotations) to prevent overfitting. as well as to help with small datasets.

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes to predict: {class_names}")

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

Classes to predict: ['Ariel_Sharon', 'Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Hugo_Chavez', 'Jacques_Chirac', 'Jean_Chretien', 'John_Ashcroft', 'Junichiro_Koizumi', 'Serena_Williams', 'Tony_Blair']


Build the CNN Model stage

this is basically the architecture of deep learning starting here

In [ ]:
print("--- Building the CNN Model ---")

model = keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Rescaling(1./255),
    data_augmentation,

    # Block 1
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),

    # Block 2
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),

    # Block 3
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),

    # Classification Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

--- Building the CNN Model ---


Compile stage

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 25, 25, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,359,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,454,220 (9.36 MB)

 Trainable params: 2,454,220 (9.36 MB)

 Non-trainable params: 0 (0.00 B)

Training stage

In [ ]:
epochs = 25
print(f"--- Training for {epochs} epochs ---")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


--- Training for 25 epochs ---
Epoch 1/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 23s 511ms/step - accuracy: 0.2761 - loss: 2.3987 - val_accuracy: 0.3654 - val_loss: 2.2359
Epoch 2/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 497ms/step - accuracy: 0.3332 - loss: 2.1705 - val_accuracy: 0.3654 - val_loss: 2.1990
Epoch 3/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 509ms/step - accuracy: 0.3467 - loss: 2.1361 - val_accuracy: 0.3654 - val_loss: 2.1198
Epoch 4/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 23s 594ms/step - accuracy: 0.3530 - loss: 2.1055 - val_accuracy: 0.3686 - val_loss: 2.0763
Epoch 5/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 38s 519ms/step - accuracy: 0.3398 - loss: 2.0522 - val_accuracy: 0.3878 - val_loss: 1.9849
Epoch 6/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 27s 689ms/step - accuracy: 0.3284 - loss: 2.0119 - val_accuracy: 0.4359 - val_loss: 1.8143
Epoch 7/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 553ms/step - accuracy: 0.3265 - loss: 1.9343 - val_accuracy: 0.4423 - val_loss: 1.6389
Epoch 8/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 523ms/step - accuracy: 0

KeyboardInterrupt: 

RESULTS AND EVALUATION

Evaluating Model Performance

Plot Accuracy and Loss Graphs for training and validation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('lfw_results.png')
print("Saved graphs to lfw_results.png")
plt.show()

Detailed Metrics for classification reports.

In [ ]:
print("--- Generating Report ---")
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(labels.numpy())

print("\n--- Classification Report ---")

print(classification_report(y_true, y_pred, target_names=class_names))